In [6]:
import pickle
import pandas as pd
from gensim.corpora.dictionary import Dictionary

In [8]:
with open('out/count_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [9]:
with open('out/lda_model.pkl', 'rb') as g:
    lda_model = pickle.load(g)

In [10]:
# This is the vocabulary
feature_names = vectorizer.get_feature_names_out()

num_top_words = 50

topic_words = []
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}:")
    top_words_idx = topic.argsort()[:-num_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(", ".join(top_words))
    topic_words.append(top_words)

Topic 1:
new, york, city, year, nyc, mayor, ny, child, jersey, theater, brooklyn, restaurant, manhattan, island, state, connecticut, food, long, number, street, bronx, michael, day, play, school, open, week, blasio, lottery, bloomberg, say, art, square, guide, mental, family, come, public, event, citys, parent, center, look, cultural, times, east, disorder, old, childhood, yearold
Topic 2:
new, city, york, coronavirus, year, real, state, estate, home, housing, travel, ny, ncov, park, space, area, building, safety, hurricane, residential, gas, water, change, oil, andrew, global, cuomo, energy, center, climate, pandemic, long, work, hotel, say, time, traffic, india, manhattan, national, week, vacation, accident, storm, service, million, leave, warming, road, administration
Topic 3:
united, states, trump, government, president, politic, election, party, donald, state, washington, house, republican, obama, international, say, presidential, ukraine, democratic, senate, biden, military, russ

In [13]:
texts = vectorizer.get_feature_names_out()

df = pd.read_csv("processced text.csv")

dictionary = Dictionary(df['preprocessed_text'].apply(lambda x: list(x)))

In [15]:
from gensim.models.coherencemodel import CoherenceModel

coherence_scores = {}
for topic_idx, words in enumerate(top_words):
    coherence_model_lda = CoherenceModel(
        topics=words, texts=df['preprocessed_text'], dictionary=dictionary, coherence='c_v'
    )
    coherence_scores[topic_idx] = coherence_model_lda.get_coherence()

AttributeError: 'dict' object has no attribute 'token2id'